# Inicialização

In [1]:
import tensorflow as tf

2022-05-28 00:34:31.176072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-28 00:34:31.176126: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


# Conectando com servidor

In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
codigoGrupo = "Neg-100"
codigoPeriodo = "11"
codigoRegra = "t11s17v"

url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemConjuntoTreino?idPeriodo={}&codigoGrupoAcao={}&codigoRegraProjecao={}".format(codigoPeriodo,codigoGrupo,codigoRegra)

In [5]:
print(url)

http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemConjuntoTreino?idPeriodo=11&codigoGrupoAcao=Neg-100&codigoRegraProjecao=t11s17v


In [6]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [7]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [8]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')

In [9]:
X = np.stack(dfTratadoX.values)

In [10]:
X.shape

(5280, 1800)

In [11]:
Y = np.stack(df['campoY'].values)

In [12]:
Y = Y.reshape(-1,1)

In [13]:
Y.shape

(5280, 1)

# Minha Rede Simples

In [14]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1800,)),
    tf.keras.layers.Dense(2000, activation=tf.nn.relu), 
    tf.keras.layers.Dense(1000, activation=tf.nn.relu), 
    tf.keras.layers.Dense(500, activation=tf.nn.relu), 
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])


2022-05-28 00:39:08.020609: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-28 00:39:08.031632: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-05-28 00:39:08.155374: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-28 00:39:11.266784: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14400000 exceeds 10% of free system memory.
2022-05-28 00:39:11.648623: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14400000 exceeds 10% of free system memory.
2022-05-28 00:39:

In [15]:
model.build()

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              3602000   
                                                                 
 dense_1 (Dense)             (None, 1000)              2001000   
                                                                 
 dense_2 (Dense)             (None, 500)               500500    
                                                                 
 dense_3 (Dense)             (None, 10)                5010      
                                                                 
Total params: 6,108,510
Trainable params: 6,108,510
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dados de Validação

In [18]:
urlValida = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemConjuntoDesenvolvimento?idPeriodo={}&codigoGrupoAcao={}&codigoRegraProjecao={}".format(codigoPeriodo,codigoGrupo,codigoRegra)

In [19]:
print(urlValida)

http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaos/obtemConjuntoDesenvolvimento?idPeriodo=11&codigoGrupoAcao=Neg-100&codigoRegraProjecao=t11s17v


In [20]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)

In [21]:
dfValida.head()

,campoX,campoY
0,"1.110304, 1.114927, 1.108322, 1.103038, 1.103...",0
1,"1.130732, 1.136893, 1.135524, 1.133470, 1.135...",0
2,"1.151473, 1.172511, 1.172511, 1.179523, 1.174...",0
3,"1.213515, 1.210640, 1.207045, 1.187635, 1.193...",0
4,"1.198813, 1.206973, 1.204006, 1.203264, 1.201...",0


In [22]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


(1936, 1800)

In [23]:
YValida = np.stack(dfValida['campoY'].values)


In [24]:
YValida = YValida.reshape(-1,1)

In [25]:
YValida.shape

(1936, 1)

In [26]:
history = model.fit(x=X,y=Y, validation_data=(XValida, YValida) ,
                    steps_per_epoch=100,
                    epochs=15,
                    validation_steps=50,
                    verbose=2)

Epoch 1/15
100/100 - 147s - loss: 1.1259 - accuracy: 0.7269 - val_loss: 0.3204 - val_accuracy: 0.9143 - 147s/epoch - 1s/step
Epoch 2/15
100/100 - 208s - loss: 0.5403 - accuracy: 0.7697 - val_loss: 0.3572 - val_accuracy: 0.9143 - 208s/epoch - 2s/step
Epoch 3/15
100/100 - 153s - loss: 0.5302 - accuracy: 0.7699 - val_loss: 0.4178 - val_accuracy: 0.9143 - 153s/epoch - 2s/step
Epoch 4/15
100/100 - 170s - loss: 0.5303 - accuracy: 0.7695 - val_loss: 0.3555 - val_accuracy: 0.9143 - 170s/epoch - 2s/step
Epoch 5/15
100/100 - 170s - loss: 0.5359 - accuracy: 0.7682 - val_loss: 0.3077 - val_accuracy: 0.9143 - 170s/epoch - 2s/step
Epoch 6/15
100/100 - 219s - loss: 0.5298 - accuracy: 0.7712 - val_loss: 0.2873 - val_accuracy: 0.9143 - 219s/epoch - 2s/step
Epoch 7/15
100/100 - 206s - loss: 0.5296 - accuracy: 0.7706 - val_loss: 0.3232 - val_accuracy: 0.9143 - 206s/epoch - 2s/step
Epoch 8/15
100/100 - 204s - loss: 0.5293 - accuracy: 0.7697 - val_loss: 0.3683 - val_accuracy: 0.9143 - 204s/epoch - 2s/step


In [27]:
# assign location
path='pesos/rede1'
 
# save
model.save_weights(path)